## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        df = pd.read_csv(input_data)
        engine = sqlite3.connect('cleaning_expenses_records.db')
        df.to_sql('expenses', con=engine, index = False)
    except Exception as e:
        print(f"Error: {e}")

data = "/content/cleaning_expenses.csv"
add_data_to_database(data)

In [3]:
 # Loading SQL extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///cleaning_expenses_records.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

#### 1. View the first five rows

In [6]:
%%sql
SELECT * FROM expenses
LIMIT 5;

 * sqlite:///cleaning_expenses_records.db
Done.


date,category,description,amount
2020-01-05,Supplies,Cleaning products,150
2020-01-08,Labor,Wages for cleaners,300
2020-01-12,Marketing,Online advertising (Facebook),50
2020-01-15,Supplies,Trash bags and gloves,25
2020-01-20,Labor,Wages for cleaners,425


## Question 2: **Answer**

In [7]:
%%sql
-- Calculate total labor costs per month
WITH labor_costs AS (
    SELECT
    STRFTIME('%m', date) AS month,
    SUM(amount) AS total_labor_cost
FROM expenses
WHERE category = 'Labor'
GROUP BY month
),
-- Find the minimum labor cost across all months
min_labor_cost AS (
    SELECT MIN(total_labor_cost) AS min_cost FROM labor_costs
)
--  Retrieve the month(s) with the lowest labor cost
SELECT
    lc.month,
    lc.total_labor_cost,

    -- Calculate percentage contribution of the minimum labor cost month
    (ROUND(lc.total_labor_cost/(SELECT CAST(SUM(total_labor_cost) AS REAL)
                               FROM labor_costs), 2) * 100)
    AS percentage_share
FROM labor_costs lc
JOIN min_labor_cost ml
ON lc.total_labor_cost = ml.min_cost;

 * sqlite:///cleaning_expenses_records.db
Done.


month,total_labor_cost,percentage_share
03,275,11.0


## Question 3 : **Answer**

In [8]:
%%sql
CREATE VIEW IF NOT EXISTS marketing_expenses_summary AS
SELECT
    description,
    SUM(amount) AS total_marketing_costs
FROM expenses
GROUP BY description;

 * sqlite:///cleaning_expenses_records.db
Done.


[]

In [9]:
%%sql
SELECT
    ROUND(SUM(total_marketing_costs)) AS total_spent,

    -- Calculating online advertising percentage share
    ROUND(SUM(CASE WHEN description LIKE 'Online advertising (Facebook)'
                   THEN total_marketing_costs ELSE 0 END) * 100.0 /
          SUM(total_marketing_costs), 2) AS online_advertising_percentage,

    -- Calculating email marketing percentage share
    ROUND(SUM(CASE WHEN description LIKE 'Email marketing campaign'
                   THEN total_marketing_costs ELSE 0 END) * 100.0 /
          SUM(total_marketing_costs), 2) AS email_marketing_percentage
FROM marketing_expenses_summary
WHERE description LIKE 'Online advertising (Facebook)'
      OR description LIKE 'Email marketing campaign';

 * sqlite:///cleaning_expenses_records.db
Done.


total_spent,online_advertising_percentage,email_marketing_percentage
125.0,40.0,60.0


## Question 4: **Answer**

In [10]:
%%sql
SELECT
    STRFTIME('%m', date) AS month,
    SUM(amount) AS cleaning_supply_expense
FROM expenses
WHERE category = 'Supplies' AND STRFTIME('%m', date) = '02';

 * sqlite:///cleaning_expenses_records.db
Done.


month,cleaning_supply_expense
02,90


## Question 5: **Answer**

In [11]:
%%sql
SELECT
    SUM(total_costs) AS total_selected_costs,
    ROUND(SUM(total_costs) * 100.0 / (SELECT SUM(amount) FROM expenses), 2) || '%'
    AS percentage_share
FROM (
    SELECT
        category,
        SUM(amount) AS total_costs,
        RANK() OVER (ORDER BY SUM(amount) DESC) AS category_rank
    FROM expenses
    GROUP BY category
) AS ranked_costs
WHERE category_rank IN (1, 3, 5);

 * sqlite:///cleaning_expenses_records.db
Done.


total_selected_costs,percentage_share
3160,73.23%
